# Sagemaker deploy

In [1]:
%%capture
!~/anaconda3/envs/python38/bin/python -m pip install sagemaker

In [2]:
from sagemaker import Session, get_execution_role
from sagemaker.model import Model
import os
import json

# Build and push docker image

In [3]:
stage = 'prod'
os.environ["staging"] = stage

In [4]:
!cd .. && ./scripts/build_and_push.sh $staging

Loging to ECR
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Selected stage:  prod
Sending build context to Docker daemon  113.3MB
Step 1/29 : FROM continuumio/miniconda3
 ---> 45461d36cbf1
Step 2/29 : WORKDIR /opt/ml/code
 ---> Using cache
 ---> 1d42458c2aad
Step 3/29 : RUN apt-get update && apt-get install -y --no-install-recommends     curl     gcc      mono-mcs     build-essential     nginx     ca-certificates     wget     pkg-config
 ---> Using cache
 ---> 4cd8dd321ea2
Step 4/29 : RUN cd /tmp &&     wget http://download.redis.io/redis-stable.tar.gz &&     tar xvzf redis-stable.tar.gz &&     cd redis-stable &&     make &&     make install
 ---> Using cache
 ---> 15d9fd4ab548
Step 5/29 : ARG conda_env=python38
 ---> Using cache
 ---> 35c119ef298c
Step 6/29 : ARG py_ver=3.8.10
 ---> Using cac

# Get Configurations Values

In [5]:
sagemaker_session = Session()
sagemaker_role = get_execution_role()

In [6]:
account = sagemaker_session.boto_session.client('sts').get_caller_identity()['Account']
region = sagemaker_session.boto_session.region_name

In [7]:
ec2_client = sagemaker_session.boto_session.client('ec2')

# Get subnets
subnets = ec2_client.describe_subnets(
    Filters=[
        {
            'Name':'owner-id',
            'Values':[account]
        }
    ]
)
# Choose just the private subnets routing to Natgateway
subnets_ids = [subnets_["SubnetId"] for subnets_ in subnets["Subnets"] if 'Tags' in subnets_.keys() and 'p' in subnets_["Tags"][0]["Value"]]

In [8]:
# Get Security Groups
security_groups = ec2_client.describe_security_groups(
    Filters=[
        {
            "Name":"owner-id",
            "Values":[account]
        },
        {
            "Name":"group-name",
            "Values":["launch-wizard-1"]
        }
    ]
)
sec_groups_ids = [sec_groups_["GroupId"] for sec_groups_ in security_groups["SecurityGroups"]]

# Tar Local Model Directory

In [9]:
import tarfile
import os

base_directory = '/'.join(os.getcwd().split('/')[:-1])
local_model_directory = os.path.join(base_directory, 'model')
local_artifacts_directory = os.path.join(base_directory, 'artifacts')
print("Directory where local model artifacts are stored " + local_model_directory)

Directory where local model artifacts are stored /home/ec2-user/SageMaker/sage-maker/model


In [10]:
def flatten(tarinfo):
    tarinfo.name = os.path.basename(tarinfo.name)
    return tarinfo

tar_name = 'model.tar.gz'
with tarfile.open(tar_name, mode='w:gz') as tar_artifacts:
    for artifact_file in os.listdir(local_model_directory):
        tar_artifacts.add(os.path.join(local_model_directory, artifact_file),
                          filter=flatten)
tar_artifacts.close()

os.rename(
    os.path.join(base_directory, 'notebooks', tar_name),
    os.path.join(local_artifacts_directory, tar_name)
)

# Upload To S3 bucket project

In [11]:
S3_Bucket = "sagemaker-us-west-2-256305374409"
File_Project = "Koombea_Blogs_Reco"

model_input = sagemaker_session.upload_data(
    path='{}/{}'.format(local_artifacts_directory, tar_name), # Path in this notebook where the data is located
    bucket=S3_Bucket,
    key_prefix='{}/{}'.format(File_Project, 'model'))

In [12]:
model_input

's3://sagemaker-us-west-2-256305374409/Koombea_Blogs_Reco/model/model.tar.gz'

# Image Uri Name

In [13]:
image_name = '{0}.dkr.ecr.{1}.amazonaws.com/blogs-reco-system'.format(account, region)

# See all the configurations

prod: blogsreco-stage-prod-2020-07-27-20-19-34 <br/>
dev: blogsreco-stage-dev-2020-07-27-19-34-26

In [14]:
import json
from time import gmtime, strftime
#endpoint_name = "blogsreco-stage-{0}-{1}".format(stage, strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
if stage == 'dev':
    endpoint_name = "blogsreco-stage-dev-2020-07-27-19-34-26" # dev
else:
    endpoint_name = "blogsreco-stage-prod-2020-07-27-20-19-34" # prod
model_name = 'model-blogsreco-stage-{0}-{1}'.format(stage, strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
print("Endpoint Name: "+endpoint_name)
print("Model Name: "+model_name)
# print("Path to model artifacts "+model_input)
# print("Image Uri to inference docker app "+image_name)
print("Subnets ids : "+json.dumps(subnets_ids, indent=True))
print("Security grousp ids: "+json.dumps(sec_groups_ids, indent=True))

Endpoint Name: blogsreco-stage-prod-2020-07-27-20-19-34
Model Name: model-blogsreco-stage-prod-2023-01-27-19-49-56
Subnets ids : [
 "subnet-027f61eee464e12db",
 "subnet-06d298c2c73bc19c2",
 "subnet-0e6ae500c242d1234",
 "subnet-0adba2c223284fabb",
 "subnet-0f789348e3a0d94cf",
 "subnet-001fbdb3812fed3db",
 "subnet-0776a5c90121609e8",
 "subnet-0ca110411c0e42d25",
 "subnet-0ee3997af1cc54fed",
 "subnet-04e17649f653d300c",
 "subnet-0e36733aecddd9521"
]
Security grousp ids: [
 "sg-093d444a974077472"
]


In [15]:
sagemaker_model = Model(model_data=model_input,
                       role=sagemaker_role,
                       sagemaker_session=sagemaker_session,
                       name=model_name,
                       vpc_config={"Subnets":[subnets_ids[2],subnets_ids[3],subnets_ids[4]],
                                  "SecurityGroupIds":sec_groups_ids},
                       image_uri=image_name)

In [16]:
sagemaker_model.create(
    instance_type="ml.t2.xlarge"
)

In [18]:
# sagemaker_model.deploy(
#    initial_instance_count=1, 
#    instance_type="ml.t2.2xlarge", 
#    endpoint_name= endpoint_name,
    #update_endpoint=True
#)

In [19]:
# sagemaker_model.delete_model()

# Get Predictor or use boto3 to make predictions

In [20]:
from sagemaker.predictor import Predictor

In [21]:
predictor = Predictor(endpoint_name=endpoint_name,
                             sagemaker_session=sagemaker_session,
                             content_type='application/json',
                             )

content_type is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


## Update Endpoint

In [22]:
predictor.update_endpoint(initial_instance_count=1,
                         instance_type="ml.t2.2xlarge",
                         model_name=sagemaker_model.name)

-------!

In [23]:
from datetime import datetime
import boto3

runtime = boto3.client('runtime.sagemaker')
start_time = datetime.now()
data = {"slug": "what-is-machine-learning", "lang":"en", "topn": 5}
data = json.dumps(data)
response = runtime.invoke_endpoint(EndpointName= endpoint_name,
                                       ContentType='application/json',
                                       Body=data)
end_time = datetime.now()
prediction = response["Body"].read().decode()
time_micro_secs = (end_time - start_time).microseconds

relates = json.loads(prediction)
print(json.dumps(relates, indent=True))

print(f" Time Take : {time_micro_secs//1000} ms", flush=True, end='\r')

{
 "relates": [
  "deep-learning-vs-machine-learning",
  "understanding-machine-learning",
  "5-uses-of-machine-learning-for-web-and-mobile-apps",
  "deep-learning-applications-for-fintech",
  "deep-learning-solutions"
 ]
}


In [24]:
print(json.dumps(response["ResponseMetadata"], indent=True))

{
 "RequestId": "9a65f175-15fa-4ede-891c-19fd31474c2c",
 "HTTPStatusCode": 200,
 "HTTPHeaders": {
  "x-amzn-requestid": "9a65f175-15fa-4ede-891c-19fd31474c2c",
  "x-amzn-invoked-production-variant": "AllTraffic",
  "date": "Fri, 27 Jan 2023 20:00:10 GMT",
  "content-type": "application/json",
  "content-length": "202"
 },
 "RetryAttempts": 0
}


In [25]:
response.keys()

dict_keys(['ResponseMetadata', 'ContentType', 'InvokedProductionVariant', 'Body'])

# Now configure API-Gateway and Lambda Function

- Go to Api-Gateway (create one)
- Go to IAM policies (create one that have log access and invoke sagemaker runtime access)
- Go to Lambda (create one and attach the previous created rol to this lambda)
- Generate code that generate CORS HEADERS and act as a proxy between api gateway call and sagemaker endpoint call